# Case 2: Understand auto type deduction.

Meyers, Scott. Effective Modern C++ . O'Reilly Media. Kindle Edition. 

##  Key idea:
 
    Deducing types for auto is the same as deducing types for templates (with only one curious exception).


In [1]:
#include <boost/type_index.hpp>
#include <iostream>

In [2]:
template<typename T>                 // conceptual template for
void func_for_x(T param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}          // deducing x's type

template<typename T>                 // conceptual template for
void func_for_cx(const T param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}   // deducing cx's type

template<typename T>                 // conceptual template for
void func_for_rx(const T& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}  // deducing rx's type
                                  
void someFunc(int, double) {}  // someFunc is a function;
                               // type is void(int, double)

In [3]:
{
  auto x = 27;                // case 3 (x is neither ptr nor reference)
  
  const auto cx = x;          // case 3 (cx isn't either)

  const auto& rx = x;         // case 1 (rx is a non-universal ref.)

  auto&& uref1 = x;           // x is int and lvalue,
                              // so uref1's type is int&

  auto&& uref2 = cx;          // cx is const int and lvalue
                              // so uref2's type is const int&

  auto&& uref3 = 27;          // 27 is int and rvalue,
                              // so uref3's type is int&&


  func_for_x(27);             // conceptual call: param's
                              // deduced type is x's type

  func_for_cx(x);             // conceptual call: param's
                              // deduced type is cx's type

  func_for_rx(x);             // conceptual call: param's
                              // deduced type is rx's type

  const char name[] =         // name's type is const char[13]
    "R. N. Briggs";

  auto arr1 = name;           // arr1's type is const char*

  auto& arr2 = name;          // arr2's type is
                              // const char (&)[13]

  auto func1 = someFunc;      // func1's type is
                              // void (*)(int, double)

  auto& func2 = someFunc;     // func2's type is
                              // void (&)(int, double)

}

T     = int
param = int
T     = int
param = int const
T     = int
param = int const&


##  Key idea:
 
    The treatment of braced initializers is the only way in which auto type
    deduction and template type deduction differ.


In [4]:
#include <initializer_list>

template<typename T>  // template with parameter
void f(T param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}    // declaration equivalent to
                      // x's declaration

template<typename T>
void f2(std::initializer_list<T> param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}

In [5]:
{
    int x1 = 27;
    int x2(27);
    int x3 = {27};
    int x4{27};
}

In [6]:
{
    auto x1 = 27;    // type is int, value is 27
    auto x2(27);     // ditto
    auto x3 = {27};  // type is std::initializer_list<int>,
                     // value is {27}
    auto x4{27};     // ditto

    //auto x5 = {1, 2, 3.0};  // error! can't deduce T for
    //                        // std::initializer_list<T>
}

In [11]:
{
    auto x = { 11, 23, 9 };  // x's type is
                             // std::initializer_list<int>

    //f({ 11, 23, 9 });        // error! can't deduce type for T

    f2({ 11, 23, 9 });        // T deduced as int, and initList's
                              // type is std::initializer_list<int>
}

T     = int
param = std::initializer_list<int>


##  Key ideas:
 
    1. A function with an auto return type that returns a braced initializer list won't compile.
 
    2. When auto is used in a parameter type specification in a C++14 lambda expression, things won't compile.


In [8]:
#include <vector>

auto createInitList()
{
  //return {1, 2, 3};    // error: can't deduce type
  //                     // for {1, 2, 3}
}

In [9]:
{
  std::vector<int> v;

  auto resetV =
    [&v](const auto& newValue) { v = newValue; };  // C++14

  //resetV( {1, 2, 3} );  // error! can't deduce type
  //                      // for { 1, 2, 3 }
}

                               Things to Remember
                               ------------------

* auto type deduction is usually the same as template type deduction, but auto
  type deduction assumes that a braced initializer represents a
  std::initializer_list, and template type deduction doesn't.

* auto in a function return type or a lambda parameter implies template type
  deduction, not auto type deduction.

